## EfficientNet
##### References : 
##### https://learnopencv.com/efficientnet-theory-code/
##### https://github.com/lukemelas/EfficientNet-PyTorch
##### https://learnopencv.com/how-to-convert-a-model-from-pytorch-to-tensorrt-and-speed-up-inference/
##### https://github.com/spmallick/learnopencv/tree/master/PyTorch-ONNX-TensorRT

In [1]:
#Import Packages

import json
from PIL import Image

import torch
from torchvision import transforms

In [2]:
model=torch.hub.load('rwightman/gen-efficientnet-pytorch', 'efficientnet_b2', pretrained=True, scriptable=True)

Using cache found in /home/arv/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master


In [3]:
!wget https://github.com/qubvel/efficientnet/raw/master/misc/panda.jpg

--2021-04-01 13:25:51--  https://github.com/qubvel/efficientnet/raw/master/misc/panda.jpg
Resolving github.com (github.com)... 13.250.177.223
Connecting to github.com (github.com)|13.250.177.223|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/qubvel/efficientnet/master/misc/panda.jpg [following]
--2021-04-01 13:25:52--  https://raw.githubusercontent.com/qubvel/efficientnet/master/misc/panda.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116068 (113K) [image/jpeg]
Saving to: ‘panda.jpg.1’

panda.jpg.1         100%[===================>] 113.35K  --.-KB/s    in 0.003s  

2021-04-01 13:25:52 (36.4 MB/s) - ‘panda.jpg.1’ saved [116068/116068]



In [4]:
img=Image.open('panda.jpg')

In [5]:
!wget https://raw.githubusercontent.com/lukemelas/EfficientNet-PyTorch/master/examples/simple/labels_map.txt

--2021-04-01 13:25:58--  https://raw.githubusercontent.com/lukemelas/EfficientNet-PyTorch/master/examples/simple/labels_map.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31565 (31K) [text/plain]
Saving to: ‘labels_map.txt.1’

labels_map.txt.1    100%[===================>]  30.83K  --.-KB/s    in 0.005s  

2021-04-01 13:25:58 (6.07 MB/s) - ‘labels_map.txt.1’ saved [31565/31565]



In [6]:
labels_map=json.load(open('labels_map.txt'))
labels_map=[labels_map[str(i)] for i in range(1000)]

In [7]:
image_size=224
tfms=transforms.Compose([transforms.Resize(image_size),
                         transforms.CenterCrop(image_size),
                         transforms.ToTensor(),
                         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [8]:
img=tfms(img).unsqueeze(0)

model.eval()
with torch.no_grad():
    logits=model(img)
preds=torch.topk(logits,k=5).indices.squeeze(0).tolist()

In [9]:
for idx in preds:
    label=labels_map[idx]
    prob=torch.softmax(logits,dim=1)[0,idx].item()
    print('{:<75} ({:.2f}%)'.format(label, prob*100))


giant panda, panda, panda bear, coon bear, Ailuropoda melanoleuca           (95.41%)
lesser panda, red panda, panda, bear cat, cat bear, Ailurus fulgens         (0.09%)
ice bear, polar bear, Ursus Maritimus, Thalarctos maritimus                 (0.06%)
earthstar                                                                   (0.05%)
badger                                                                      (0.04%)


In [10]:
#Convert to ONNX format
ONNX_PATH='efficientnetb2.onnx'
torch.onnx.export(model,img,ONNX_PATH,input_names=['input'],
                 output_names=['output'],export_params=True)

In [11]:
import onnx
onnx_model=onnx.load(ONNX_PATH)
onnx.checker.check_model(onnx_model)

In [49]:
#from torch2trt import torch2trt

ModuleNotFoundError: No module named 'torch2trt'

In [12]:
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np

In [13]:
import tensorrt as trt


In [14]:
TRT_LOGGER=trt.Logger()

In [23]:
def build_engine(onnx_file_path):
    builder=trt.Builder(TRT_LOGGER)
    network=builder.create_network()
    parser=trt.OnnxParser(network,TRT_LOGGER)
    
    builder.max_workspace_size=1<<20
    builder.max_batch_size=1
    if builder.platform_has_fast_fp16:
        builder.fp16_mode=True
    
    with open(onnx_file_path,'rb') as model:
        parser.parse(model.read())
    
    engine=builder.build_cuda_engine(network)
    #context=engine.create_execution_context()
    
    return engine#,context

In [24]:
def main():
    #engine,context=build_engine(ONNX_PATH)
    engine=build_engine(ONNX_PATH)
    for binding in engine:
        if engine.binding_is_input(binding):
            input_shape=engine.get_binding_shape(binding)
            input_size=trt.volume(input_shape)*engine.max_batch_size*np.dtype(np.float32).itemsize
            device_input=cuda.mem_alloc(input_size)
        else:
            output_shape=engine.get_binding_shape(binding)
            host_output=cuda.pagelocked_empty(trt.volume(output_shape)* engine.max_batch_size, dtype=np.float)
            device_output=cuda_mem_alloc(host_output.nbytes)
            
    stream=cuda.Stream()
    
    host_input=np.array(img)
    cuda.memcpy_htod_async(device_input,device_output,stream)
    
    stream.synchronize()
    
    output_data=torch.Tensor(host_output).reshape(engine.max_batch_size, output_shape[0])

    
main()    
    

TypeError: 'NoneType' object is not iterable